### *IPCC SR15 scenario assessment*

<img style="float: right; height: 100px; margin-top: 10px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Descriptive statistics of electricity generation

This notebook computes indicators and diagnostics of the primary-energy timeseries by fuel
for **Table 2.7** in the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
cats_15 = specs.pop('cats_15')
cats_15_no_lo = specs.pop('cats_15_no_lo')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
years = [2020, 2030, 2050]

In [ ]:
df = sr1p5.filter(category=cats_15, year=years)

## Initialize a `pyam.Statistics` instance

In [ ]:
stats = pyam.Statistics(df=df,
                        filters=[('all 1.5', {}),
                                 ('no & lo os 1.5', {'category': cats_15_no_lo}),
                                 ('hi os 1.5', {'category': ['1.5C high overshoot']})
                                ], rows=True)

In [ ]:
header='Electricity generation (EJ)'
header_share='Share in electricity generation (%)'
header_growth='Growth (factor)'

statistics_settings = dict(
    header=header,
    header_share='Share in electricity generation (%)',
    header_growth='Growth (factor)',
    growth_year=2050,
    base_year=2020
)

In [ ]:
def add_statistics(data, base, row, growth_year, base_year,
                   header, header_share, header_growth):
    stats.add(data, header=header, row=row)
    if base is not None:
        stats.add(data / base * 100, header=header_share, row=row)
    stats.add(data[growth_year] / data[base_year] - 1,
              header=header_growth, row=row,
              subheader='{}-{}'.format(base_year, growth_year))

## Extract total electricity generation timeseries and add summary statistics

In [ ]:
se = df.filter(variable='Secondary Energy|Electricity').timeseries()
se.index = se.index.droplevel([2, 3, 4])

In [ ]:
add_statistics(se, None, 'total generation', **statistics_settings)

## Compute share of renewables by various types in electricity generation

Only use scenarios for this indicator that report both biomass and the aggregate non-biomass timeseries - otherwise, the share would be distorted.

### All renewables (biomass and non-biomass)

In [ ]:
df_pe_res = df.filter()
df_pe_res.require_variable('Secondary Energy|Electricity|Non-Biomass Renewables', exclude_on_fail=True)
df_pe_res.require_variable('Secondary Energy|Electricity|Biomass', exclude_on_fail=True)
df_pe_res.filter(exclude=False, inplace=True)

In [ ]:
res = (
    df_pe_res.filter(variable=['Secondary Energy|Electricity|Biomass',
                               'Secondary Energy|Electricity|Non-Biomass Renewables'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(res, se, 'renewables', **statistics_settings)

### Biomass

In [ ]:
res_bio = (
    df_pe_res.filter(variable=['Secondary Energy|Electricity|Biomass'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(res_bio, se, 'biomass', **statistics_settings)

### Non-biomass renewables

In [ ]:
res_non_bio = (
    df_pe_res.filter(variable=['Secondary Energy|Electricity|Non-Biomass Renewables'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(res_non_bio, se, 'non-biomass', **statistics_settings)

### Renewable energy from wind and solar

As above, verify that scenarios report values for both 'Wind' and 'Solar' 

In [ ]:
df_win_sol = df.filter()
df_win_sol.require_variable('Secondary Energy|Electricity|Solar', exclude_on_fail=True)
df_win_sol.require_variable('Secondary Energy|Electricity|Wind', exclude_on_fail=True)
df_win_sol.filter(exclude=False, inplace=True)

In [ ]:
win_sol = (
    df_win_sol.filter(variable=['Secondary Energy|Electricity|Solar',
                                'Secondary Energy|Electricity|Wind '])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(win_sol, se, 'wind & solar', **statistics_settings)

## Compute share of nuclear in electricity generation

In [ ]:
nuc = (
    df.filter(variable=['Secondary Energy|Electricity|Nuclear'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(nuc, se, 'nuclear', **statistics_settings)

## Compute share of fossil in electricity generation

In [ ]:
df_fossil = df.filter()
df_fossil.require_variable('Secondary Energy|Electricity|Coal', exclude_on_fail=True)
df_fossil.require_variable('Secondary Energy|Electricity|Gas', exclude_on_fail=True)
df_fossil.require_variable('Secondary Energy|Electricity|Oil', exclude_on_fail=True)
df_fossil.filter(exclude=False, inplace=True)

In [ ]:
fossil = (
    df.filter(variable=['Secondary Energy|Electricity|Coal',
                        'Secondary Energy|Electricity|Gas',
                        'Secondary Energy|Electricity|Oil'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(fossil, se, 'fossil', **statistics_settings)

In [ ]:
coal = (
    df_fossil.filter(variable=['Secondary Energy|Electricity|Coal'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(coal, se, 'coal', **statistics_settings)

In [ ]:
gas = (
    df_fossil.filter(variable=['Secondary Energy|Electricity|Gas'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(gas, se, 'gas', **statistics_settings)

In [ ]:
oil = (
    df_fossil.filter(variable=['Secondary Energy|Electricity|Oil'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(oil, se, 'oil', **statistics_settings)

## Display and export summary statistics for all 1.5°C pathways to `xlsx`

In [ ]:
summary = (
    stats.summarize(center='median', fullrange=True)
    .reindex(columns=['count', header, header_share, header_growth], level=0)
)
summary

In [ ]:
summary.to_excel('output/table_2.7_electricity_generation.xlsx')